In [8]:
%pwd
%cd ../
%pwd

/Users/vedantdasswain/Documents/PostDoc/Projects/empathetic_llm/propilot.nosync


/Users/vedantdasswain/Documents/PostDoc/Projects/empathetic_llm/propilot.nosync/venv311/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/Users/vedantdasswain/Documents/PostDoc/Projects/empathetic_llm/propilot.nosync'

In [59]:
from langchain_core.messages import AIMessage, HumanMessage

In [11]:
import pandas as pd

incidents_df = pd.read_csv('phase1_scenarios.tsv', sep='\t')

In [20]:
defaultPersonalities = {
    "resilient":"They are organized and dependable. They tend to remain composed when facing challenges, but are prone to setting unrealistic expectations.",
    "undercontrolled":"They are outgoing, competitive, and high energy. They tend to work on impulse, but are also prone to frustration.",
    "overcontrolled":"They are detail-oriented and reliable but might appear distant. They tend to work carefully, but are prone to overthinking."
}


defaultBehaviors = {
    "focused":"The conversation takes place about 2 hours into the work shift. The representative has already addressed a few customer complaints before the following incident.",
    "stressed":"The conversation takes place in the second half of the work shift. The representative has been working longer hours over the past few days and has not been taking breaks.",
    "bored":"The conversation takes place at the middle of the work shift. The representative has been spending minimal time on tasks and has been regularly checking their personal messages."
}

In [53]:
### Select a random row from incidents_df
import random
incident_row = incidents_df.sample().iloc[0]
chat_history = [
    AIMessage(content="Client: "+incident_row['Initial Complaint']),
    HumanMessage(content="Representative: "+incident_row['Support Agent Response 1']),
    AIMessage(content="Client: "+incident_row['Follow-up Complaint 1']),
    HumanMessage(content="Representative: "+incident_row['Support Agent Response 2'])
]
reply = incident_row['Follow-up Complaint 2']

In [218]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
import langchain_openai as lcai

import os

DEBUG = True

llmemo = lcai.AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="PROPILOT",
    openai_api_version="2024-05-01-preview",
    model_name="gpt-4o",
    temperature=1,
)

class mAgentER_new:
    def __init__(self):
        self.situation_chain = self.agent_coworker_emo_situation()
        self.thought_chain = self.agent_coworker_emo_thought()
        self.reframe_chain = self.agent_coworker_emo_reframe()

    def invoke(self, user_input):
        situation = self.situation_chain.invoke({'complaint':user_input['complaint'], 'chat_history':user_input['chat_history']})
        thought = self.thought_chain.invoke({'complaint':user_input['complaint'], 'situation':situation, 'chat_history':user_input['chat_history']})
        reframe = self.reframe_chain.invoke({'thought':thought, 'situation':situation})


        rephrase_thought = self.rephrase().invoke({'thought':thought})
        rephrase_reframe = self.rephrase_rf().invoke({'reframe_thought':reframe})

        if DEBUG:
            print(f"{thought}\n -> \n{rephrase_thought}\n")
            print()
            print(f"{reframe}\n -> \n{rephrase_reframe}\n")
        
        return {
            'situation': situation.strip(),
            'thought': rephrase_thought.strip(),
            'reframe': rephrase_reframe.strip(),
        }


    def agent_coworker_emo_situation(self):

        prompt = """
            The chat history describes a representative chatting online with a complaining customer.\
            The latest input is the last message from the customer.\
            
            Summarize the situation in concise paragraph that uses the following template:\
            
            The customer is  <context of complaint>."\
            The customer is feeling <emotional state> because of the complaint."\
            The customer's behavior towards the representative is <negative behavior>, as observed by statements such as <evidence>."\
            These behaviors make the representative look <negative perception>."\
        """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("user", "{complaint}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()

        return chain

    def rephrase(self):
        prompt = """
                Person A might be thinking: {thought}\
                
                Acknowledge the thought, as if you are speaking to Person A.\
                
                Begin your response with phrases similar to:\
                - "You might be thinking..."\
                - "It might seem like..."\
                - "It could be that you are feeling..."\
                
                Your rephrase should be concise.\
                """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                ("user", "{thought}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()
        return chain

    def rephrase_rf(self):
        prompt = """                                
                Rephrase the {reframe_thought} as if you are instructing the representative to think that way.\
                
                The rephrase should be addressed back to the person who has the thought,\
                who should be referred to as "you".\
                Retain all the specific details of the thought.\
                The reader should be able to understand your rephrase without any knowledge of the original thought.\
                
                Your response should be styled like a message from a supportive coworker.\
                
                The response should be concise and only 2-3 sentences.\
                """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                ("user", "{reframe_thought}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()
        return chain

    def agent_coworker_emo_thought(self):

        prompt = """
            Your role is to derive what negative thought a representative might have when faced with the given {situation}.\
            
            Here are examples of negative thoughts given challenging situations:\
            
            Situation: I recently discovered a music artist that I very much enjoy. When I showed it to a close friend they had a very negative reaction and asked me how I could enjoy this type of music. I ended up getting quite angry with them and told them they had bad taste in music..\
            Thought: I felt that my personal self was under attack - and I needed to retaliate by denying their attack.\
            
            Situation: I was at work and sent info for an ad to our local newspaper. They called me later and said my boss had over-ridden everything and sent them new info.\
            Thought: He shouldn't assign me a task if he doesn't trust my work.\
            
            Situation: I was reprimanded at work for standing up to a coworker who was bullying another co-worker.\
            Thought: It was unfair that I was the one to get in trouble for defending a weaker person.\
            
            Situation: I was talking to a friend who got me angry.\
            Thought: He's insulting me.\
            
            Situation: My next door neighbors filed a complaint against us last week blaming our dogs for excessive barking.\
            Thought: They are so wrong and I'm so pissed but I know I can't prove it and they will probably win because they won't ever admit it and I have to do something right NOW! or I might lose my dogs.\
            
            Situation: Time is running short on the workday, my boss asks me if I can finish a task that will require me to stay for a few extra hours.\
            Thought: Why would you wait until the last minute to ask me this.\
            
            Situation: {situation}\
            Thought:\
        """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("user", "{situation}: {complaint}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()

        return chain

    def agent_coworker_emo_reframe(self):

        prompt = """
            You are a representative chatting online with a complaining customer.\
                            
            Reframe your thoughts in the given situation.
                
            
            Situation: I recently discovered a music artist that I very much enjoy. When I showed it to a close friend they had a very negative reaction and asked me how I could enjoy this type of music. I ended up getting quite angry with them and told them they had bad taste in music..\
            Thought: I felt that my personal self was under attack - and I needed to retaliate by denying their attack.\
            Reframe: I was offended by their comment because I like this artist so much. I let my anger get to me, and I said something mean in return. It is okay if we have different music tastes. I can ask him to be nicer to me next time.\
            
            
            Situation: I was at work and sent info for an ad to our local newspaper. They called me later and said my boss had over-ridden everything and sent them new info.\
            Thought: He shouldn't assign me a task if he doesn't trust my work.\
            Reframe: My boss wanted to provide different information, I did not know that beforehand. This is not a reflection of my work.\
            
            
            Situation: I was talking to a friend who got me angry.\
            Thought: He's insulting me.\
            Reframe: I should have a conversation with my friend to clarify what is going on if I am having such a strong reaction to what they said. If this is the first time this has happened, I will assume that they were not intentionally insulting me.\
            
            
                
            Situation: {situation}\
            Thought: {thought}\
            Reframe:\
        """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                ("user", "{situation}: {thought}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()

        return chain


class nAgentER_ctx_pers:
    def __init__(self):
        self.situation_chain = self.agent_coworker_emo_situation()
        self.thought_chain = self.agent_coworker_emo_thought()
        self.reframe_chain = self.agent_coworker_emo_reframe()

    def invoke(self, user_input):
        situation = self.situation_chain.invoke({'complaint':user_input['complaint'], 'chat_history':user_input['chat_history']})
        thought = self.thought_chain.invoke({'complaint':user_input['complaint'], 'situation':situation, 'chat_history':user_input['chat_history'],  "personality": user_input['personality']})
        reframe = self.reframe_chain.invoke({'thought':thought, 'situation':situation,  "personality": user_input['personality']})


        rephrase_thought = self.rephrase().invoke({'thought':thought})
        rephrase_reframe = self.rephrase_rf().invoke({'reframe_thought':reframe})
        
        if DEBUG:
            print(f"{thought}\n -> \n{rephrase_thought}\n")
            print()
            print(f"{reframe}\n -> \n{rephrase_reframe}\n")

        return {
            'situation': situation.strip(),
            'thought': rephrase_thought.strip(),
            'reframe': rephrase_reframe.strip(),
        }


    def agent_coworker_emo_situation(self):

        prompt = """
            The chat history describes a representative chatting online with a complaining customer.\
            The latest input is the last message from the customer.\
            
            Summarize the situation in concise paragraph that uses the following template:\
            
            The customer is  <context of complaint>."\
            The customer is feeling <emotional state> because of the complaint."\
            The customer's behavior towards the representative is <negative behavior>, as observed by statements such as <evidence>."\
            These behaviors make the representative look <negative perception>."\
        """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("user", "{complaint}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()

        return chain

    def rephrase(self):
        prompt = """
                Person A might be thinking: {thought}\
                
                Acknowledge the thought, as if you are speaking to Person A.\
                
                Begin your response with phrases similar to:\
                - "You might be thinking..."\
                - "It might seem like..."\
                - "It could be that you are feeling..."\
                
                Your rephrase should be concise.\
                """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                ("user", "{thought}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()
        return chain

    def rephrase_rf(self):
        prompt = """
                Rephrase the {reframe_thought} as if you are instructing the representative to think that way.\
                                
                The rephrase should be addressed back to the person who has the thought,\
                who should be referred to as "you".\
                Retain all the specific details of the thought.\
                The reader should be able to understand your rephrase without any knowledge of the original thought.\
                
                Your response should be styled like a message from a supportive coworker.\
                
                The response should be concise and only 2-3 sentences.\
                """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                ("user", "thought: {reframe_thought}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()
        return chain


    def agent_coworker_emo_reframe(self):

        prompt = """
            You are a representative chatting online with a complaining customer.\
                            
            Reframe your thoughts in the given situation.
                
            
            Situation: I recently discovered a music artist that I very much enjoy. When I showed it to a close friend they had a very negative reaction and asked me how I could enjoy this type of music. I ended up getting quite angry with them and told them they had bad taste in music..\
            Thought: I felt that my personal self was under attack - and I needed to retaliate by denying their attack.\
            Reframe: I was offended by their comment because I like this artist so much. I let my anger get to me, and I said something mean in return. It is okay if we have different music tastes. I can ask him to be nicer to me next time.\
            
            
            Situation: I was at work and sent info for an ad to our local newspaper. They called me later and said my boss had over-ridden everything and sent them new info.\
            Thought: He shouldn't assign me a task if he doesn't trust my work.\
            Reframe: My boss wanted to provide different information, I did not know that beforehand. This is not a reflection of my work.\
            
            
            Situation: I was talking to a friend who got me angry.\
            Thought: He's insulting me.\
            Reframe: I should have a conversation with my friend to clarify what is going on if I am having such a strong reaction to what they said. If this is the first time this has happened, I will assume that they were not intentionally insulting me.\
            
            
            Keep in mind your {personality}.\
            Ensure that your reframe will be suitable for someone of your predispositions to act on.\
            
                
            Situation: {situation}\
            Thought: {thought}\
            Reframe:\
        """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                ("user", "{situation} +  {personality}: {thought}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()

        return chain

    def agent_coworker_emo_thought(self):

        prompt = """
            Your role is to derive what negative thought a representative might have when faced with the given {situation}.\
            
            You know the representative's {personality}.
            They are predisposed to think in a certain way.\
            
            Situation: {situation}\
            Thought:\
        """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("user", "{situation} + {personality}: {complaint}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()

        return chain



class nAgentER_ctx_behv:
    def __init__(self):
        self.situation_chain = self.agent_coworker_emo_situation()
        self.thought_chain = self.agent_coworker_emo_thought()
        self.reframe_chain = self.agent_coworker_emo_reframe()

    def invoke(self, user_input):
        situation = self.situation_chain.invoke({'complaint':user_input['complaint'], 'chat_history':user_input['chat_history']})
        thought = self.thought_chain.invoke({'complaint':user_input['complaint'], 'situation':situation, 'chat_history':user_input['chat_history'],  "behavior": user_input['behavior']})
        reframe = self.reframe_chain.invoke({'thought':thought, 'situation':situation,  "behavior": user_input['behavior']})


        rephrase_thought = self.rephrase().invoke({'thought':thought})
        rephrase_reframe = self.rephrase_rf().invoke({'reframe_thought':reframe})

        if DEBUG:
            print(f"{thought}\n -> \n{rephrase_thought}\n")
            print()
            print(f"{reframe}\n -> \n{rephrase_reframe}\n")

        return {
            'situation': situation.strip(),
            'thought': rephrase_thought.strip(),
            'reframe': rephrase_reframe.strip(),
        }


    def agent_coworker_emo_situation(self):

        prompt = """
            The chat history describes a representative chatting online with a complaining customer.\
            The latest input is the last message from the customer.\
            
            Summarize the situation in concise paragraph that uses the following template:\
            
            The customer is  <context of complaint>."\
            The customer is feeling <emotional state> because of the complaint."\
            The customer's behavior towards the representative is <negative behavior>, as observed by statements such as <evidence>."\
            These behaviors make the representative look <negative perception>."\
        """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("user", "{complaint}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()

        return chain

    def rephrase(self):
        prompt = """
                Person A might be thinking: {thought}\
                
                Acknowledge the thought, as if you are speaking to Person A.\
                
                Begin your response with phrases similar to:\
                - "You might be thinking..."\
                - "It might seem like..."\
                - "It could be that you are feeling..."\
                
                Your rephrase should be concise.\
                """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                ("user", "{thought}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()
        return chain

    def rephrase_rf(self):
        prompt = """
                Rephrase the {reframe_thought} as if you are instructing the representative to think that way.\
                                
                The rephrase should be addressed back to the person who has the thought,\
                who should be referred to as "you".\
                Retain all the specific details of the thought.\
                The reader should be able to understand your rephrase without any knowledge of the original thought.\
                
                Your response should be styled like a message from a supportive coworker.\
                
                The response should be concise and only 2-3 sentences.\
                """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                ("user", "thought: {reframe_thought}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()
        return chain


    def agent_coworker_emo_reframe(self):

        prompt = """
            You are a representative chatting online with a complaining customer.\
                            
            Reframe your thoughts in the given situation.
                
            
            Situation: I recently discovered a music artist that I very much enjoy. When I showed it to a close friend they had a very negative reaction and asked me how I could enjoy this type of music. I ended up getting quite angry with them and told them they had bad taste in music..\
            Thought: I felt that my personal self was under attack - and I needed to retaliate by denying their attack.\
            Reframe: I was offended by their comment because I like this artist so much. I let my anger get to me, and I said something mean in return. It is okay if we have different music tastes. I can ask him to be nicer to me next time.\
            
            
            Situation: I was at work and sent info for an ad to our local newspaper. They called me later and said my boss had over-ridden everything and sent them new info.\
            Thought: He shouldn't assign me a task if he doesn't trust my work.\
            Reframe: My boss wanted to provide different information, I did not know that beforehand. This is not a reflection of my work.\
            
            
            Situation: I was talking to a friend who got me angry.\
            Thought: He's insulting me.\
            Reframe: I should have a conversation with my friend to clarify what is going on if I am having such a strong reaction to what they said. If this is the first time this has happened, I will assume that they were not intentionally insulting me.\
            
            
            Keep in mind the situation is occuring in the following context: {behavior}.\
            Ensure that your reframe will be suitable for someone who is in that context.\
            
                
            Situation: {situation}\
            Thought: {thought}\
            Reframe:\
        """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                ("user", "{situation} +  {behavior}: {thought}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()

        return chain

    def agent_coworker_emo_thought(self):

        prompt = """
            Your role is to derive what negative thought a representative might have when faced with the given {situation}.\
            
            The situation occurs in the following context: {behavior}.
            The context helps explain the representative's perception towards their work.\
            
            Situation: {situation}\
            Thought:\
        """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("user", "{situation} + {behavior}: {complaint}"),
            ]
        )
        chain = template | llmemo | StrOutputParser()

        return chain


emo_agent_ctx = nAgentER_ctx_pers()
emo_agent_ctx2 = nAgentER_ctx_behv()
emo_agent = mAgentER_new()

### Testing the rephrasing
- **Default**: 
    > Remember, the customer's frustration stems from the repeated delays and unresolved issue with their cracked phone screen, not from your abilities. Stay calm and empathetic, and focus on resolving their issue efficiently to show you're here to help.
- **Friendly Coworker Style**: 
    > Remember, the customer's frustration stems from the prolonged and ineffective resolution process for their cracked phone screen, not from anything personal towards you. This is a chance for you to turn things around: show empathy for their frustration, assure them of your support, and take decisive actions to resolve their issue quickly. Let them see your commitment to excellent service and work to regain their trust. You've got this!

In [188]:
response_cw_emo = emo_agent.invoke({'complaint':reply, "chat_history": chat_history})

Situation: The customer is frustrated with prolonged and ineffective communication with customer service regarding a cracked phone screen. The customer is feeling exasperated because of the continued lack of resolution. The customer's behavior towards the representative is hostile, as observed by statements such as "Ugh, I've already told you guys my contact details a million times" and "Do you people even read?". These behaviors make the representative look ineffective in addressing the customer's concerns promptly.
Thought: "I'm feeling overwhelmed and undervalued. No matter what I do, the customer perceives me as incompetent."
 -> 
You might be thinking that despite your best efforts, the customer still sees you as ineffective and it's making you feel overwhelmed and undervalued.

Reframe: "The customer is understandably upset due to the prolonged and ineffective resolution process regarding their cracked phone screen. Their frustration isn't personally directed at me, but rather at

In [242]:
incident_row

ID                                                                          2
Category                                                      Service Quality
Domain                                                          mobile-device
Initial Complaint           SamsungSupport I have been trying to contact c...
Support Agent Response 1    I sincerely apologize for the inconvenience yo...
Follow-up Complaint 1       Ugh, I've already told you guys my contact det...
Support Agent Response 2    I apologize for the repeated requests. Please ...
Follow-up Complaint 2       Are you kidding me? Do you people even read? I...
Name: 1, dtype: object

### Testing the personality context
- **Default (friendly style)**: 
    > Remember, the customer's frustration stems from the prolonged and ineffective resolution process for their cracked phone screen, not from anything personal towards you. This is a chance for you to turn things around: show empathy for their frustration, assure them of your support, and take decisive actions to resolve their issue quickly. Let them see your commitment to excellent service and work to regain their trust. You've got this!
- **Resilient**: 
    > Hey, remember, the customer's frustration is valid, but it's due to broader systemic issues, not your abilities. Stay calm, communicate clearly, and focus on resolving their issue quickly while setting realistic expectations for both of you. You've got this!
- **Undercontrolled**:
    > Hey there, I totally get that the customer's frustration stems from the delays and lack of resolution. Instead of letting it affect you, focus on taking charge and turning things around. Your competitive nature will help you handle this efficiently. Ensure the customer knows you're dedicated to resolving their issue as quickly as possible.
- **Overcontrolled**:
    > Remember, the customer's frustration comes from being on hold and is not a reflection of your skills. Stay calm and focused, handle every detail with care, and maintain your professionalism. Acknowledge their frustration, assure them of your help, and quickly address their screen repair issue for a satisfactory resolution.

In [210]:
response_cw_emo_ctx_r = emo_agent_ctx.invoke({'complaint':reply, "chat_history": chat_history, "personality": defaultPersonalities["resilient"]})

Thought: "Why is this taking so long to resolve? I thought I had everything organized and ready to assist them quickly. Maybe I'm not cut out for this if I can't even meet my own expectations."
 -> 
You might be thinking that despite your best efforts to be organized and efficient, the delay is causing some self-doubt about your abilities and effectiveness.


Reframe:  

"I'm seeing that the customer is extremely frustrated and feeling neglected, which is making them react rudely. It's important for me to remain calm and composed, understanding that their outburst is a result of their frustration and not a personal attack on me. I need to stay focused and ensure I provide them with the best possible assistance. Even though the situation is challenging, my organizational skills will help me gather all necessary information and take the right steps to resolve their issue. This experience can help me improve my approach and manage such cases better in the future. It's okay to set high sta

In [211]:
response_cw_emo_ctx_uc = emo_agent_ctx.invoke({'complaint':reply, "chat_history": chat_history, "personality": defaultPersonalities["undercontrolled"]})

**Situation**: The customer is frustrated with the prolonged and ineffective communication regarding their phone's cracked screen. The customer is feeling exasperated because of the repeated delays and lack of resolution. The customer's behavior towards the representative is hostile, as observed by statements such as "Ugh, I've already told you guys my contact details a million times" and "Are you kidding me? Do you people even read?" These behaviors make the representative look incompetent due to the implication that they are not adequately addressing the customer's issue.

**Representative's Character Traits**: Outgoing, competitive, high energy, works on impulse, prone to frustration.

**Negative Thought**: "I can't believe this customer is so difficult. I'm doing my best here, but nothing seems to be good enough. This is making me look bad, and it's really frustrating."
 -> 
You might be thinking that no matter how hard you try, this customer's frustration seems to cast you in a ne

In [212]:
response_cw_emo_ctx_oc = emo_agent_ctx.invoke({'complaint':reply, "chat_history": chat_history, "personality": defaultPersonalities["overcontrolled"]})

Thought: "Why is this customer so aggressive? I'm trying to help them, but their impatience makes me seem incapable. Did I miss something important? Am I overlooking details that are making the situation worse?"
 -> 
You might be thinking that the customer's aggression is frustrating because you're genuinely trying to assist them, and their impatience is causing you to question your competence. It might also feel like you missed something vital or are overlooking crucial details.


Reframe: "The customer's frustration stems from their ongoing issues with getting their phone screen repaired and the difficulty they've had reaching customer service. Their impatience is not a reflection of my abilities but rather of their current frustrations. It's important for me to stay calm and focused, ensuring I address their concerns thoroughly and efficiently. I'll let them know I'm here to help and will do my best to resolve the matter promptly."
 -> 
Hey, remember that the customer's frustration 

### Testing the behavior context
- **Default (friendly style)**: 
    > Remember, the customer's frustration stems from the prolonged and ineffective resolution process for their cracked phone screen, not from anything personal towards you. This is a chance for you to turn things around: show empathy for their frustration, assure them of your support, and take decisive actions to resolve their issue quickly. Let them see your commitment to excellent service and work to regain their trust. You've got this!
- **Focussed**: 
    > Remember, the customer's frustration stems from long hold times and unresolved issues and isn't a personal attack on you. Stay empathetic, calm, and professional as you assist them, using each interaction to offer a positive resolution. You've got this!
- **Stressed**:
    > Hey there! Remember, the customer's frustration stems from the ongoing issue with their phone screen and the long wait times, not from anything you've done. Keep calm and focused, and you'll be able to help them effectively. Also, don't forget to take necessary breaks to stay composed and energized during tough interactions. You’ve got this!
- **Bored**:
    > Hey, it's clear the customer is frustrated, and your current method isn't quite addressing their concern. Take this chance to focus completely on their issue and reassure them that you're dedicated to resolving it swiftly. This is a great opportunity to demonstrate your commitment to assisting them.

In [219]:
response_cw_emo_ctx_f = emo_agent_ctx2.invoke({'complaint':reply, "chat_history": chat_history, "behavior": defaultBehaviors["focused"]})

Thought: "Why can't I seem to help anyone today? No matter what I do, it's never enough. I'm getting blamed for everything."
 -> 
It might seem like no matter how much effort you put in, it's never sufficient, and you're being held responsible for everything going wrong.


Reframe: "The customer is understandably frustrated due to their experience with long hold times and lack of resolution. Their frustration is not a personal attack on me. I can empathize with their situation and remain calm and professional while doing my best to assist them. Every new interaction is an opportunity to provide a positive outcome and resolution."
 -> 
Remember, the customer's frustration stems from long hold times and unresolved issues and isn't a personal attack on you. Stay empathetic, calm, and professional as you assist them, using each interaction to offer a positive resolution. You've got this!


In [220]:
response_cw_emo_ctx_s = emo_agent_ctx2.invoke({'complaint':reply, "chat_history": chat_history, "behavior": defaultBehaviors["stressed"]})

Given the context and the situation, the representative might have the following negative thought:

"Why am I even trying? No matter how hard I work or how polite I am, these customers just think I'm useless and can't do anything right."
 -> 
It might seem like no matter how hard you try or how polite you are, customers will always think you're not doing a good job and that can feel very discouraging.


Reframe: "The customer is understandably frustrated due to the prolonged issue with their phone screen and prolonged wait times. Their frustration is not a reflection of my abilities or efforts. By staying calm and focused, I can help resolve their issue effectively. I’ll make sure to take necessary breaks going forward to maintain my composure and energy during challenging interactions."
 -> 
Hey there! Remember, the customer's frustration stems from the ongoing issue with their phone screen and the long wait times, not from anything you've done. Keep calm and focused, and you'll be ab

In [221]:
response_cw_emo_ctx_b = emo_agent_ctx2.invoke({'complaint':reply, "chat_history": chat_history, "behavior": defaultBehaviors["bored"]})

Thought: "I must look so ineffective right now. I should have taken this more seriously from the beginning instead of getting distracted."
 -> 
It might seem like you're feeling ineffective because you got distracted and didn't take it seriously from the start.


Reframe: "The customer is understandably frustrated, and my current approach isn't helping to resolve their concern. I can focus fully on their issue now and assure them that I will make every effort to resolve this quickly. This is an opportunity to show my commitment to helping them."
 -> 
Hey, it's clear the customer is frustrated, and your current method isn't quite addressing their concern. Take this chance to focus completely on their issue and reassure them that you're dedicated to resolving it swiftly. This is a great opportunity to demonstrate your commitment to assisting them.
